# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [5]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [6]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [7]:
def draw_landmarks(image, results):// vẽ các đốt và nối các đốt với nhau
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [8]:
def draw_styled_landmarks(image, results)://chỉnh sửa style cho đẹp hơn
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=0),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=0)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(168,229,132), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(216,229,132), thickness=2, circle_radius=3)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(229,145,132), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(145,132,229), thickness=2, circle_radius=3)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(229,145,132), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(145,132,229), thickness=2, circle_radius=3)
                             )

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)//luôn luôn mở opencv để 

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break//ấn q để không show cái của opencv
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
len(results.left_hand_landmarks.landmark)

In [ ]:
results

In [12]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [ ]:
len(results.left_hand_landmarks.landmark)

In [15]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [16]:
// mỗi điểm chấm bé có ba giá trị x y z , phần thân có giá trị visibility để xác định có xuất hiện hay ko
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)// một thân có 33 landmark  x 4
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)// 468 landmark
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [ ]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() 
    if results.face_landmarks 
    else np.zeros(1404)


In [18]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh]) // trích ra 1662 landmark

In [19]:
result_test = extract_keypoints(results)

In [ ]:
result_test

In [21]:
468*3+33*4+21*3+21*3

1662

In [22]:
np.save('0', result_test)

In [23]:
np.load('0.npy')

array([ 0.50381893,  0.42118058, -1.08082664, ...,  0.        ,
        0.        ,  0.        ])

# 4. Setup Folders for Collection

In [24]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [ ]:
# hello
## 0
## 1
## 2
## ...
## 29
# thanks

# I love you

In [25]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [26]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [27]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [28]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [29]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [30]:
np.array(sequences).shape

(90, 30, 1662)

In [31]:
np.array(labels).shape

(90,)

In [32]:
X = np.array(sequences)

In [33]:
X.shape

(90, 30, 1662)

In [34]:
y = to_categorical(labels).astype(int)

In [35]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [37]:
y_test.shape

(5, 3)

# 7. Build and Train LSTM Neural Network

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [39]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [40]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))//giảm số chiều lstm
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [41]:
res = [.7, 0.2, 0.1]

In [42]:
actions[np.argmax(res)]

'hello'

In [43]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [44]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 3s 272ms/step - loss: 1.2984 - categorical_accuracy: 0.3327
Epoch 2/2000
3/3 [==============================] - 0s 92ms/step - loss: 1.3700 - categorical_accuracy: 0.2466
Epoch 3/2000
3/3 [==============================] - 0s 83ms/step - loss: 2.2083 - categorical_accuracy: 0.2818
Epoch 4/2000
3/3 [==============================] - 0s 66ms/step - loss: 2.2056 - categorical_accuracy: 0.3503
Epoch 5/2000
3/3 [==============================] - 0s 64ms/step - loss: 3.1684 - categorical_accuracy: 0.3542
Epoch 6/2000
3/3 [==============================] - 0s 64ms/step - loss: 1.7071 - categorical_accuracy: 0.3347
Epoch 7/2000
3/3 [==============================] - 0s 65ms/step - loss: 1.0360 - categorical_accuracy: 0.5578
Epoch 8/2000
3/3 [==============================] - 0s 66ms/step - loss: 0.8751 - categorical_accuracy: 0.6732
Epoch 9/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.7557 - categorical_accuracy: 0.6321


3/3 [==============================] - 0s 76ms/step - loss: 0.1468 - categorical_accuracy: 0.9472
Epoch 75/2000
3/3 [==============================] - 0s 73ms/step - loss: 0.0996 - categorical_accuracy: 0.9550
Epoch 76/2000
3/3 [==============================] - 0s 73ms/step - loss: 0.0994 - categorical_accuracy: 0.9726
Epoch 77/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.0583 - categorical_accuracy: 0.9765
Epoch 78/2000
3/3 [==============================] - 0s 71ms/step - loss: 0.0494 - categorical_accuracy: 0.9882
Epoch 79/2000
3/3 [==============================] - 0s 66ms/step - loss: 0.1042 - categorical_accuracy: 0.9550
Epoch 80/2000
3/3 [==============================] - 0s 68ms/step - loss: 0.0231 - categorical_accuracy: 1.0000
Epoch 81/2000
3/3 [==============================] - 0s 69ms/step - loss: 0.0318 - categorical_accuracy: 0.9902
Epoch 82/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.0334 - categorical_accuracy: 0.9824
Epoch 

3/3 [==============================] - 0s 64ms/step - loss: 1.5596e-04 - categorical_accuracy: 1.0000
Epoch 146/2000
3/3 [==============================] - 0s 64ms/step - loss: 1.4078e-04 - categorical_accuracy: 1.0000
Epoch 147/2000
3/3 [==============================] - 0s 64ms/step - loss: 1.3478e-04 - categorical_accuracy: 1.0000
Epoch 148/2000
3/3 [==============================] - 0s 65ms/step - loss: 1.5074e-04 - categorical_accuracy: 1.0000
Epoch 149/2000
3/3 [==============================] - 0s 63ms/step - loss: 1.0282e-04 - categorical_accuracy: 1.0000
Epoch 150/2000
3/3 [==============================] - 0s 61ms/step - loss: 1.4811e-04 - categorical_accuracy: 1.0000
Epoch 151/2000
3/3 [==============================] - 0s 63ms/step - loss: 1.1243e-04 - categorical_accuracy: 1.0000
Epoch 152/2000
3/3 [==============================] - 0s 64ms/step - loss: 1.1747e-04 - categorical_accuracy: 1.0000
Epoch 153/2000
3/3 [==============================] - 0s 65ms/step - loss: 1.35

3/3 [==============================] - 0s 71ms/step - loss: 5.0462e-05 - categorical_accuracy: 1.0000
Epoch 216/2000
3/3 [==============================] - 0s 69ms/step - loss: 4.1827e-05 - categorical_accuracy: 1.0000
Epoch 217/2000
3/3 [==============================] - 0s 71ms/step - loss: 4.9538e-05 - categorical_accuracy: 1.0000
Epoch 218/2000
3/3 [==============================] - 0s 68ms/step - loss: 4.8201e-05 - categorical_accuracy: 1.0000
Epoch 219/2000
3/3 [==============================] - 0s 71ms/step - loss: 4.5630e-05 - categorical_accuracy: 1.0000
Epoch 220/2000
3/3 [==============================] - 0s 70ms/step - loss: 4.7418e-05 - categorical_accuracy: 1.0000
Epoch 221/2000
3/3 [==============================] - 0s 71ms/step - loss: 4.3714e-05 - categorical_accuracy: 1.0000
Epoch 222/2000
3/3 [==============================] - 0s 69ms/step - loss: 3.7163e-05 - categorical_accuracy: 1.0000
Epoch 223/2000
3/3 [==============================] - 0s 72ms/step - loss: 3.97

3/3 [==============================] - 0s 69ms/step - loss: 1.8714e-05 - categorical_accuracy: 1.0000
Epoch 286/2000
3/3 [==============================] - 0s 70ms/step - loss: 1.7835e-05 - categorical_accuracy: 1.0000
Epoch 287/2000
3/3 [==============================] - 0s 71ms/step - loss: 1.6929e-05 - categorical_accuracy: 1.0000
Epoch 288/2000
3/3 [==============================] - 0s 80ms/step - loss: 1.7333e-05 - categorical_accuracy: 1.0000
Epoch 289/2000
3/3 [==============================] - 0s 119ms/step - loss: 1.4131e-05 - categorical_accuracy: 1.0000
Epoch 290/2000
3/3 [==============================] - 0s 104ms/step - loss: 1.5497e-05 - categorical_accuracy: 1.0000
Epoch 291/2000
3/3 [==============================] - 0s 116ms/step - loss: 1.5328e-05 - categorical_accuracy: 1.0000
Epoch 292/2000
3/3 [==============================] - 0s 108ms/step - loss: 1.6127e-05 - categorical_accuracy: 1.0000
Epoch 293/2000
3/3 [==============================] - 0s 67ms/step - loss: 

...

3/3 [==============================] - 0s 72ms/step - loss: 6.6581e-07 - categorical_accuracy: 1.0000
Epoch 845/2000
3/3 [==============================] - 0s 71ms/step - loss: 6.7884e-07 - categorical_accuracy: 1.0000
Epoch 846/2000
3/3 [==============================] - 0s 67ms/step - loss: 5.8105e-07 - categorical_accuracy: 1.0000
Epoch 847/2000
3/3 [==============================] - 0s 70ms/step - loss: 5.6801e-07 - categorical_accuracy: 1.0000
Epoch 848/2000
3/3 [==============================] - 0s 70ms/step - loss: 6.8954e-07 - categorical_accuracy: 1.0000
Epoch 849/2000
3/3 [==============================] - 0s 71ms/step - loss: 9.2679e-07 - categorical_accuracy: 1.0000
Epoch 850/2000
3/3 [==============================] - 0s 72ms/step - loss: 7.3191e-07 - categorical_accuracy: 1.0000
Epoch 851/2000
3/3 [==============================] - 0s 69ms/step - loss: 7.9989e-07 - categorical_accuracy: 1.0000
Epoch 852/2000
3/3 [==============================] - 0s 69ms/step - loss: 6.61

3/3 [==============================] - 0s 68ms/step - loss: 4.4857e-07 - categorical_accuracy: 1.0000
Epoch 915/2000
3/3 [==============================] - 0s 75ms/step - loss: 5.7779e-07 - categorical_accuracy: 1.0000
Epoch 916/2000
3/3 [==============================] - 0s 73ms/step - loss: 5.0887e-07 - categorical_accuracy: 1.0000
Epoch 917/2000
3/3 [==============================] - 0s 72ms/step - loss: 6.6952e-07 - categorical_accuracy: 1.0000
Epoch 918/2000
3/3 [==============================] - 0s 72ms/step - loss: 6.2808e-07 - categorical_accuracy: 1.0000
Epoch 919/2000
3/3 [==============================] - 0s 68ms/step - loss: 4.6206e-07 - categorical_accuracy: 1.0000
Epoch 920/2000
3/3 [==============================] - 0s 70ms/step - loss: 4.2015e-07 - categorical_accuracy: 1.0000
Epoch 921/2000
3/3 [==============================] - 0s 70ms/step - loss: 4.2131e-07 - categorical_accuracy: 1.0000
Epoch 922/2000
3/3 [==============================] - 0s 70ms/step - loss: 5.58

3/3 [==============================] - 0s 73ms/step - loss: 3.5618e-07 - categorical_accuracy: 1.0000
Epoch 985/2000
3/3 [==============================] - 0s 70ms/step - loss: 5.1683e-07 - categorical_accuracy: 1.0000
Epoch 986/2000
3/3 [==============================] - 0s 71ms/step - loss: 4.3324e-07 - categorical_accuracy: 1.0000
Epoch 987/2000
3/3 [==============================] - 0s 71ms/step - loss: 4.8400e-07 - categorical_accuracy: 1.0000
Epoch 988/2000
3/3 [==============================] - 0s 98ms/step - loss: 4.5349e-07 - categorical_accuracy: 1.0000
Epoch 989/2000
3/3 [==============================] - 0s 84ms/step - loss: 5.0332e-07 - categorical_accuracy: 1.0000
Epoch 990/2000
3/3 [==============================] - 0s 81ms/step - loss: 4.8376e-07 - categorical_accuracy: 1.0000
Epoch 991/2000
3/3 [==============================] - 0s 76ms/step - loss: 5.6805e-07 - categorical_accuracy: 1.0000
Epoch 992/2000
3/3 [==============================] - 0s 74ms/step - loss: 3.56

3/3 [==============================] - 0s 76ms/step - loss: 4.3871e-07 - categorical_accuracy: 1.0000
Epoch 1054/2000
3/3 [==============================] - 0s 75ms/step - loss: 4.3242e-07 - categorical_accuracy: 1.0000
Epoch 1055/2000
3/3 [==============================] - 0s 76ms/step - loss: 3.7071e-07 - categorical_accuracy: 1.0000
Epoch 1056/2000
3/3 [==============================] - 0s 76ms/step - loss: 2.8084e-07 - categorical_accuracy: 1.0000
Epoch 1057/2000
3/3 [==============================] - 0s 86ms/step - loss: 3.7677e-07 - categorical_accuracy: 1.0000
Epoch 1058/2000
3/3 [==============================] - 0s 78ms/step - loss: 3.7584e-07 - categorical_accuracy: 1.0000
Epoch 1059/2000
3/3 [==============================] - 0s 79ms/step - loss: 4.0121e-07 - categorical_accuracy: 1.0000
Epoch 1060/2000
3/3 [==============================] - 0s 75ms/step - loss: 4.0540e-07 - categorical_accuracy: 1.0000
Epoch 1061/2000
3/3 [==============================] - 0s 76ms/step - lo

KeyboardInterrupt: 

In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

# 8. Make Predictions

In [46]:
res = model.predict(X_test)

In [47]:
actions[np.argmax(res[4])]

'thanks'

In [48]:
actions[np.argmax(y_test[4])]

'thanks'

# 9. Save Weights

In [49]:
model.save('action.h5')

In [ ]:
del model

In [50]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [51]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [52]:
yhat = model.predict(X_test)

In [53]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [54]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 1]],

       [[2, 0],
        [1, 2]],

       [[3, 1],
        [0, 1]]], dtype=int64)

In [55]:
accuracy_score(ytrue, yhat)

0.8

# 11. Test in Real Time

In [56]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
sequence.reverse()

In [ ]:
len(sequence)

In [ ]:
sequence.append('def')

In [ ]:
sequence.reverse()

In [ ]:
sequence[-30:]

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [60]:
res[np.argmax(res)] > threshold

True

In [ ]:
(num_sequences,30,1662)

In [ ]:
model.predict(np.expand_dims(X_test[0], axis=0))